# Main statistical analysis
- narrow candidate to 10 candidate using four metrics(total profit, weekly profit, monthly profit, flights frequency)


In [199]:
import pandas as pd
import numpy as np

roundTrip_profit_g = pd.read_csv('../data/temproary_data/roundTrip_profit.csv')
roundTrip_profit_g['round_trip_route_IATA'] = roundTrip_profit_g['round_trip_route_IATA'].apply(lambda x: eval(x))
roundTrip_profit_g['inbound_FL_DATE'] = pd.to_datetime(roundTrip_profit_g['inbound_FL_DATE'])
roundTrip_profit_g['outbound_FL_DATE'] = pd.to_datetime(roundTrip_profit_g['outbound_FL_DATE'])

roundTrip_fre_g = pd.read_csv('../data/temproary_data/round_trip_flights.csv')
roundTrip_fre_g['round_trip_route_IATA'] = roundTrip_fre_g['round_trip_route_IATA'].apply(lambda x: eval(x))

## Aggregrating all the factors and assign ranks to each factors for round-trip route
consider factors:
- flight frequency
- flight profit (total profit - total cost)
- TODO: weekly profit

In [200]:
# calculate the ranking of the profit
roundTrip_profit = (roundTrip_profit_g.groupby('round_trip_route_IATA')['profit']
                 .sum()
                 .reset_index()
                 .rename(columns={'profit':'round_trip_profit'}))
roundTrip_profit = (roundTrip_profit.assign(round_trip_profit_ranking = roundTrip_profit['round_trip_profit']
                                    .rank(ascending = False, method = 'min')))


# calculate the ranking of the frequency
roundTrip_fre = (roundTrip_fre_g.groupby('round_trip_route_IATA')['outbound_OCCUPANCY_RATE']
                 .count().sort_values(ascending=False)
                 .reset_index()
                 .rename(columns={'outbound_OCCUPANCY_RATE':'round_trip_flights_count'}))
roundTrip_fre = roundTrip_fre[roundTrip_fre['round_trip_route_IATA'].isin(roundTrip_profit['round_trip_route_IATA'])]
roundTrip_fre = (roundTrip_fre.assign(round_trip_flights_count_ranking = roundTrip_fre['round_trip_flights_count']
                              .rank(ascending = False, method = 'min')))

# calculate the monthly average profit
roundTrip_profit_m = (roundTrip_profit_g.assign(month = np.minimum(roundTrip_profit_g['inbound_FL_DATE'], roundTrip_profit_g['outbound_FL_DATE']).dt.month)
                      .groupby('round_trip_route_IATA')
                      .apply(lambda df: df.groupby('month')['profit'].sum().mean())
                      .reset_index()
                      .rename(columns={0:'monthly_avg_profit'}))
roundTrip_profit_m = roundTrip_profit_m[roundTrip_profit_m['round_trip_route_IATA'].isin(roundTrip_profit['round_trip_route_IATA'])]
roundTrip_profit_m = (roundTrip_profit_m.assign(monthly_avg_profit_ranking = roundTrip_profit_m['monthly_avg_profit']
                                                .rank(ascending = False, method = 'min')))

# calculate the weekly average profit
roundTrip_profit_w = (roundTrip_profit_g.assign(week = np.minimum(roundTrip_profit_g['inbound_FL_DATE'], roundTrip_profit_g['outbound_FL_DATE']).dt.isocalendar().week)
                      .groupby('round_trip_route_IATA')
                      .apply(lambda df: df.groupby('week')['profit'].sum().mean())
                      .reset_index()
                      .rename(columns={0:'weekly_avg_profit'}))
roundTrip_profit_w = roundTrip_profit_w[roundTrip_profit_w['round_trip_route_IATA'].isin(roundTrip_profit['round_trip_route_IATA'])]
roundTrip_profit_w = (roundTrip_profit_w.assign(weekly_avg_profit_ranking = roundTrip_profit_w['weekly_avg_profit']
                                                .rank(ascending = False, method = 'min')))

# calculate the ranking of the operation career
roundTrip_op = (roundTrip_profit_g
                .groupby('round_trip_route_IATA')
                .apply(lambda df: len(set(df['inbound_OP_CARRIER']).union(set(df['outbound_OP_CARRIER']))))
                .reset_index()
                .rename(columns={0:'round_trip_op_count'}))

# calcualte the proprotion of the delay flights for each round trip route
inbound_trips = roundTrip_profit_g[['inbound_DEP_DELAY', 'inbound_ARR_DELAY', 'round_trip_route_IATA']]
outbound_trips = roundTrip_profit_g[['outbound_DEP_DELAY', 'outbound_ARR_DELAY', 'round_trip_route_IATA']]

all_flights = pd.concat(
    [inbound_trips.rename(columns={'inbound_DEP_DELAY':'DEP_DELAY', 'inbound_ARR_DELAY':'ARR_DELAY'}, inplace= False),
    outbound_trips.rename(columns={'outbound_DEP_DELAY':'DEP_DELAY', 'outbound_ARR_DELAY':'ARR_DELAY'}, inplace= False)],
    axis=0
)
all_flights = all_flights[all_flights['round_trip_route_IATA'].isin(roundTrip_profit['round_trip_route_IATA'])]

all_flights['is_dep_delay'] = all_flights['DEP_DELAY'].apply(lambda val : True if val >= 15 else False if val <= -15 else 0)
all_flights['is_arr_delay'] = all_flights['ARR_DELAY'].apply(lambda val : True if val >= 15 else False if val <= -15 else 0)

roundTrip_dep_delay_rate = all_flights.groupby('round_trip_route_IATA')['is_dep_delay'].mean().sort_values(ascending=True).reset_index().rename(columns={'is_dep_delay':'dep_delay_rate'})
roundTrip_dep_delay_rate = (roundTrip_dep_delay_rate.assign(dep_delay_rate_ranking = roundTrip_dep_delay_rate['dep_delay_rate'].rank(ascending = True, method = 'min')))

roundTrip_arr_delay_rate = all_flights.groupby('round_trip_route_IATA')['is_arr_delay'].mean().sort_values(ascending=True).reset_index().rename(columns={'is_arr_delay':'arr_delay_rate'})
roundTrip_arr_delay_rate = (roundTrip_arr_delay_rate.assign(arr_delay_rate_ranking = roundTrip_arr_delay_rate['arr_delay_rate'].rank(ascending = True, method = 'min')))

# calcualate the average delay rate for each round trip route



# # merge the ranks
round_trip_info = roundTrip_profit.merge(roundTrip_fre, on='round_trip_route_IATA', how='inner')

round_trip_info = round_trip_info.merge(roundTrip_profit_m, on='round_trip_route_IATA', how='left')

round_trip_info = round_trip_info.merge(roundTrip_profit_w, on='round_trip_route_IATA', how='left')

round_trip_info = round_trip_info.merge(roundTrip_op, on='round_trip_route_IATA', how='left')

round_trip_info = round_trip_info.merge(roundTrip_dep_delay_rate, on='round_trip_route_IATA', how='left')

round_trip_info = round_trip_info.merge(roundTrip_arr_delay_rate, on='round_trip_route_IATA', how='left')




# # # aggregate the ranking
round_trip_info = round_trip_info.assign(avg_ranking = ((round_trip_info['round_trip_flights_count_ranking'] * 0.2 + 
                                                         round_trip_info['round_trip_profit_ranking'] * 0.7 + 
                                                         round_trip_info['monthly_avg_profit_ranking'] * 0.5 +
                                                         round_trip_info['weekly_avg_profit_ranking'] * 0.5) / 4))
candidate_roundTrip_route = round_trip_info.sort_values(by='avg_ranking', ascending=True).head(10)


In [201]:
candidate_roundTrip_route

,round_trip_route_IATA,round_trip_profit,round_trip_profit_ranking,round_trip_flights_count,round_trip_flights_count_ranking,monthly_avg_profit,monthly_avg_profit_ranking,weekly_avg_profit,weekly_avg_profit_ranking,round_trip_op_count,dep_delay_rate,dep_delay_rate_ranking,arr_delay_rate,arr_delay_rate_ranking,avg_ranking
1978,"(JFK, LAX)",1.065010e+08,1.0,3140,4.0,3.550033e+07,1.0,8.192385e+06,1.0,4,0.153025,924.0,0.172134,1153.0,0.625
2004,"(JFK, SFO)",4.670737e+07,8.0,1842,21.0,1.556912e+07,8.0,3.592874e+06,8.0,4,0.226384,2131.0,0.256515,2277.0,4.450
1109,"(DCA, ORD)",4.445894e+07,9.0,1764,24.0,1.481965e+07,9.0,3.419918e+06,9.0,6,0.189059,1576.0,0.213435,1801.0,5.025
128,"(ATL, CLT)",4.716123e+07,7.0,1534,43.0,1.572041e+07,7.0,3.627787e+06,7.0,3,0.124185,466.0,0.140482,584.0,5.125
1555,"(EWR, SFO)",5.828642e+07,3.0,1199,94.0,1.942881e+07,3.0,4.483570e+06,3.0,2,0.277731,2544.0,0.297331,2536.0,5.975
1093,"(DCA, LGA)",4.069849e+07,12.0,1674,35.0,1.356616e+07,12.0,3.130653e+06,12.0,2,0.222222,2081.0,0.296595,2533.0,6.850
1283,"(DFW, IAH)",3.786681e+07,15.0,1432,57.0,1.262227e+07,15.0,2.912832e+06,15.0,7,0.182961,1471.0,0.246159,2178.0,9.225
2392,"(MSP, ORD)",3.276681e+07,25.0,1705,31.0,1.092227e+07,25.0,2.520524e+06,25.0,9,0.226979,2136.0,0.248094,2199.0,12.175
1382,"(DSM, ORD)",3.788485e+07,14.0,947,161.0,1.262828e+07,14.0,2.914219e+06,14.0,8,0.240232,2280.0,0.293559,2519.0,14.000
831,"(CLT, GSP)",5.746858e+07,4.0,772,253.0,1.915619e+07,4.0,4.420660e+06,4.0,1,0.139896,703.0,0.123705,363.0,14.350
